In [79]:
import pandas as pd
import numpy as np
from graphviz import Digraph
src = "https://saedsayad.com/datasets/weather_nominal.csv"
#src = "lenses.tab"
data = pd.read_csv(src, skiprows=(1,2))

In [80]:
data

,Outlook,Temperature,Humidity,Windy,Play golf
0,Overcast,Hot,High,False,Yes
1,Sunny,Mild,High,False,Yes
2,Sunny,Cool,Normal,False,Yes
3,Sunny,Cool,Normal,True,No
4,Overcast,Cool,Normal,True,Yes
5,Rainy,Mild,High,False,No
6,Rainy,Cool,Normal,False,Yes
7,Sunny,Mild,Normal,False,Yes
8,Rainy,Mild,Normal,True,Yes
9,Overcast,Mild,High,True,Yes


In [81]:
from math import log

def entropy(*probs):
  """Calculate information entropy"""
  try:
    total = sum(probs)
    return sum([-p / total * log(p / total, 2) for p in probs])
  except:
    return 0

In [82]:
#FIRST IMPLEMENTATION
tree = Digraph()
target_ftr_name = 'Play golf'
#decision node = attribute
#chance_node = attr-val
#end node = class
def id3(dataset, decision_node = None, chance_node = None):
    prnt_val_freq = list(dataset[target_ftr_name].value_counts())
    prnt_node_entropy = entropy(*prnt_val_freq)
    #base case 1
    if prnt_node_entropy == 0:
        value_of_target = list(subset[target_ftr_name])[0]
        tree.edge('Root', value_of_target)
        return
    #base case 2 if there`s no descriptive features
    if dataset.shape[1] == 1:
        value_of_target = list(subset[target_ftr_name])[0]
        tree.edge(chance_node, value_of_target)
        return
    #if we have the best atrribute(with the highest IG) to make subsets by its values
    if decision_node:
        chance_nodes = list(set(dataset[decision_node]))
        for val in chance_nodes:
            tree.edge(decision_node, str(val))
            #label=column_name , val = value
            subset = dataset[dataset[decision_node] == val]
            #base case 3
            if subset.shape[0] == 0:
                most_common_target = dataset[target_ftr_name].value_counts().idxmax()
                tree.edge(str(val),most_common_target )
                continue
            value_freq = subset[target_ftr_name].value_counts()
            value_freq = [val for val in value_freq]
            subset_entropy = entropy(*value_freq)
            if subset_entropy == 0:
                #we can take either value of target feature
                value_of_target = list(subset[target_ftr_name])[0]
                tree.edge(str(val),  value_of_target)
                continue
            subset = subset.drop(decision_node, axis=1)
            id3(subset, chance_node = val)
    # we have to find attribute with the highest IG
    else:
        #
        ig_all = []
        for label, column in dataset.iteritems():
            #skip target feature
            if label == target_ftr_name: continue
            uv = list(set(column))
            subset_entropy = 0
            for val in uv:
                #label=column_name , val = value
                subset = dataset[dataset[label] == val]
                value_freq = subset[target_ftr_name].value_counts()
                value_freq = [val for val in value_freq]
                subset_entropy += ( len(subset)/len(dataset)) * entropy(*value_freq)
            ig = prnt_node_entropy  - subset_entropy
            ig_all.append(ig)
        #the atrribute with the highest information gain
        decision_node = dataset.columns[ig_all.index(max(ig_all))]
        # i haven't came up with something to write here
        if chance_node:
            #chance node is the attr-val
            tree.edge(chance_node, decision_node)
        id3(dataset, decision_node)
id3(data)

In [83]:
print(tree)

digraph {
	Outlook -> Rainy
	Rainy -> Humidity
	Humidity -> Normal
	Normal -> Yes
	Humidity -> High
	High -> No
	Outlook -> Overcast
	Overcast -> Yes
	Outlook -> Sunny
	Sunny -> Windy
	Windy -> False
	False -> Yes
	Windy -> True
	True -> No
}


In [85]:
#SECOND IMPLEMENTATION
#PS: i should add base cases
tree = Digraph()
target_ftr_name = 'Play golf'
#decision node = attribute
#chance_node = attr-val
#end node = class
def choose_decision_node(dataset):
    prnt_val_freq = list(dataset[target_ftr_name].value_counts())
    prnt_node_entropy = entropy(*prnt_val_freq)
    ig_all = []
    for label, column in dataset.iteritems():
        #skip target feature
        if label == target_ftr_name: continue
        uv = list(set(column))
        subset_entropy = 0
        for val in uv:
            #label=column_name , val = value
            subset = dataset[dataset[label] == val]
            value_freq = subset[target_ftr_name].value_counts()
            value_freq = [val for val in value_freq]
            subset_entropy += ( len(subset)/len(dataset)) * entropy(*value_freq)
        ig = prnt_node_entropy  - subset_entropy
        ig_all.append(ig)
    #the atrribute with the highest information gain
    decision_node = dataset.columns[ig_all.index(max(ig_all))]
    return decision_node
def build_chance_nodes_subsets(dataset, decision_node):
    chance_nodes = list(set(dataset[decision_node]))
    subsets = []
    for val in chance_nodes:
        #label=column_name , val = value
        subset = dataset[dataset[decision_node] == val]
        subsets.append( (val,subset))
    return subsets 
def prepare_for_evaluating_ie(subset):
    value_freq = subset[target_ftr_name].value_counts()
    value_freq = [val for val in value_freq]
    return value_freq
def add_space(margin):
    space = ""
    for num_val in range(margin):
        space += "\s"
    return space
def id3(dataset, decision_node = None, prnt_chance_node =None,  margin = -1):
    margin += 1
    if decision_node:
        space = add_space(margin)
        chance_nodes_subsets = build_chance_nodes_subsets(dataset, decision_node)
        for subset in chance_nodes_subsets:
            #subset = (subset_select_val, very_subset)
            subset_select_val = subset[0]
            subset = subset[1]
            class_freq_list = prepare_for_evaluating_ie(subset)
            subset_entropy = entropy(*class_freq_list)
            if subset_entropy == 0:
                #we can take either value of target feature
                value_of_target = list(subset[target_ftr_name])[0]
                print(space, "IF {} THEN class:{}".format(subset_select_val, value_of_target))
                continue
            subset = subset.drop(decision_node, axis=1)
            id3(subset, prnt_chance_node = subset_select_val, margin=margin-1)
    #if there`s no decision node
    else:
        space = add_space(margin)
        decision_node = choose_decision_node(dataset)
        if prnt_chance_node:
            print(space, "IF {} THEN Check Attribute : {}".format(prnt_chance_node, decision_node))
        else:
            print("CHECK ATTRIBUTE", decision_node)
        id3(dataset, decision_node = decision_node, margin = margin  )
id3(data)

CHECK ATTRIBUTE Outlook
\s IF Rainy THEN Check Attribute : Humidity
\s\s IF Normal THEN class:Yes
\s\s IF High THEN class:No
\s IF Overcast THEN class:Yes
\s IF Sunny THEN Check Attribute : Windy
\s\s IF False THEN class:Yes
\s\s IF True THEN class:No


In [62]:
print(tree)

digraph {
}
